In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime, timedelta

def table_scraper(station, start_date, end_date):
    current_date = start_date
    search_url = 'http://www.wunderground.com/history/daily/{}/date/{}-{}-{}'

    while current_date < end_date:
        format_search_url = search_url.format('K'+station, current_date.year, current_date.month, current_date.day)

        driver = webdriver.Chrome()
        driver.get(format_search_url)
        table = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))[-1]

        df = pd.read_html(table.get_attribute('outerHTML'))[0].dropna()[["Time", "Condition"]]
        df["Date"] = current_date

        if start_date == current_date:
            df.to_csv(f'{station}_{start_date.year}_{end_date.year}.csv', header=True, index=False, mode="w")
        else:
            df.to_csv(f'{station}_{start_date.year}_{end_date.year}.csv', header=False, index=False, mode="a")

        current_date += timedelta(days=1)


In [53]:
area = 'GAI'
start = datetime(year=2015, month=2, day=1)
end = datetime(year=2016, month=2, day=1)

table_scraper(area, start, end)

KeyError: "['Time' 'Condition'] not in index"